<a href="https://colab.research.google.com/github/jayneld/Pandas-Practice/blob/main/Copy_of_Python_pandas_Practice_Problems_UC_Berkeley_MIDS_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python pandas Practice Problems

Welcome to datascience@berkeley's collection of Python pandas practice problems.

To use this notebook, go to `File` --> `Save a copy in Drive` in the menu bar above to create a copy of this notebook on your computer. A new tab will open with the copy and you will be the owner. Your version will not be visible to anyone else.

A few notes to get you started:

* The space provided for code is not a line limit or hint. Add and use as many lines of code or additional cells as necessary.
* Lines preceded with '#' are comments and provide additional explanation but will not affect written code or execution.
* Solutions can be found in the complementary [Python pandas Practice Problems for Beginners - Solutions notebook](https://colab.research.google.com/drive/1f8PTrhgJR093fBth5Ka23KCWTIGbgPds?usp=sharing).
* Additional resources for the concepts used in each problem can be found at [the blog post here](https://ischoolonline.berkeley.edu/blog/python-pandas-practice-problems/).

\

In [1]:
# Import statements go here

import pandas as pd
import statsmodels.api as sm

## Importing Data and Making a DataFrame
The statsmodels package (installed in the code cell above) includes built-in datasets. Execute the code below to download data from the [American National Election Studies of 1996](https://www.statsmodels.org/dev/datasets/generated/anes96.html) and print a detailed description of the schema.

The next cell extracts the `Dataset` object from the submodule and saves the `DataFrame` to the variable `df`. In the questions that follow, use `df` when referencing the dataset.

In [2]:
anes96 = sm.datasets.anes96
print(anes96.NOTE)

::

    Number of observations - 944
    Number of variables - 10

    Variables name definitions::

            popul - Census place population in 1000s
            TVnews - Number of times per week that respondent watches TV news.
            PID - Party identification of respondent.
                0 - Strong Democrat
                1 - Weak Democrat
                2 - Independent-Democrat
                3 - Independent-Indpendent
                4 - Independent-Republican
                5 - Weak Republican
                6 - Strong Republican
            age : Age of respondent.
            educ - Education level of respondent
                1 - 1-8 grades
                2 - Some high school
                3 - High school graduate
                4 - Some college
                5 - College degree
                6 - Master's degree
                7 - PhD
            income - Income of household
                1  - None or less than $2,999
                2  - $3,000-$4,9

In [3]:
dataset_anes96 = anes96.load_pandas()
df = dataset_anes96.data

## 1. DataFrame Basic Properties Exercise

Our DataFrame (`df`) contains data on registered voters in the United States, including demographic information and political preference. Using `pandas`, print the first 5 rows of the DataFrame to get a sense of what the data looks like. Next, answer the following questions:


*   How many observations are in the DataFrame?
*   How many variables are measured (how many columns)?
*   What is the age of the youngest person in the data? The oldest?
*   How many days a week does the average respondent watch TV news (round to the nearest tenth)?
*   Check for missing values. Are there any?






In [4]:
# Your code here
#df.head(5)
#df.columns
#df.shape #944 observations, 11 columns
#df['age'].max() #91 max age
#df['age'].min() #19 min age
#df['TVnews'].mean().round(1) #3.7 hours of TV news watched
bool_series = pd.isnull(df["popul"])
df[bool_series]
#No missing values? I cant figure out how to show JUST missing values but it seems like none



,popul,TVnews,selfLR,ClinLR,DoleLR,PID,age,educ,income,vote,logpopul


## 2. Data Processing Exercise

We want to adjust the dataset for our use. Do the following:


*   Rename the `educ` column `education`.
*   Create a new column called `party` based on each respondent's answer to `PID`. `party` should equal `Democrat` if the respondent selected either Strong Democrat or Weak Democrat. `party` will equal `Republican` if the respondent selected Strong or Weak Republican for `PID` and `Independent` if they selected anything else.
*   Create a new column called `age_group` that buckets respondents into the following categories based on their `age`: 18-24, 25-34, 35-44, 45-54, 55-64, and 65 and over.



In [5]:
# Your code here
#rename educ column to education
df = df.rename(
    columns={
        'educ': 'education',
    }
)
df.head()
# PID - Party identification of respondent.
                #0 - Strong Democrat
                #1 - Weak Democrat
                #2 - Independent-Democrat
                #3 - Independent-Indpendent
                #4 - Independent-Republican
                #5 - Weak Republican
                #6 - Strong Republican
#function to map numerical values to party names
def map_pid(PID):
  if PID in[0,1]:
    return 'Democrat'
  elif PID in[2,3,4]:
    return 'Independent'
  else:
    return 'Republican'
#apply function to new column
df['party'] = df['PID'].apply(map_pid)

#bins are edges of each bin, so 17-24 24-34 etc
bins = [17, 24, 34, 44, 54, 64, float('inf')]
#labels correspond starting from first two bins, then bins 2-3 then bins 3-4 etc
labels =['18-24', '28-34', '35-44', '45-54', '55-64', '65 and older']
#create new columns that cuts age based on bins, right=True means that the right bin number is included
df['age_group']=pd.cut(df['age'], bins=bins, labels=labels, right=True)

df.head()

,popul,TVnews,selfLR,ClinLR,DoleLR,PID,age,education,income,vote,logpopul,party,age_group
0,0.0,7.0,7.0,1.0,6.0,6.0,36.0,3.0,1.0,1.0,-2.302585,Republican,35-44
1,190.0,1.0,3.0,3.0,5.0,1.0,20.0,4.0,1.0,0.0,5.247550,Democrat,18-24
2,31.0,7.0,2.0,2.0,6.0,1.0,24.0,6.0,1.0,0.0,3.437208,Democrat,18-24
3,83.0,4.0,3.0,4.0,5.0,1.0,28.0,6.0,1.0,0.0,4.420045,Democrat,28-34
4,640.0,7.0,5.0,6.0,4.0,0.0,68.0,6.0,1.0,0.0,6.461624,Democrat,65 and older


## 3. Filtering Data Exercise

Use the filtering method to find all the respondents who have the impression that Bill Clinton is moderate or conservative (`ClinLR` equals 4 or higher). How many respondents are in this subset?

Among these respondents, how many have a household income less than $50,000 and attended at least some college?

In [6]:
# Your code here
Clint = df[df['ClinLR'] >=4]
#print(len(Clint))  282

subset = Clint[(df['income'] <=19) & (df['education'] >=4)]
subset


<ipython-input-6-0d035b565d6d>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset = Clint[(df['income'] <=19) & (df['education'] >=4)]


,popul,TVnews,selfLR,ClinLR,DoleLR,PID,age,education,income,vote,logpopul,party,age_group
3,83.0,4.0,3.0,4.0,5.0,1.0,28.0,6.0,1.0,0.0,4.420045,Democrat,28-34
4,640.0,7.0,5.0,6.0,4.0,0.0,68.0,6.0,1.0,0.0,6.461624,Democrat,65 and older
5,110.0,3.0,3.0,4.0,6.0,1.0,21.0,4.0,1.0,0.0,4.701389,Democrat,18-24
6,100.0,7.0,5.0,6.0,4.0,1.0,77.0,4.0,1.0,0.0,4.606170,Democrat,65 and older
7,31.0,1.0,5.0,4.0,5.0,4.0,21.0,4.0,1.0,0.0,3.437208,Independent,18-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,35.0,0.0,3.0,4.0,6.0,1.0,36.0,7.0,19.0,0.0,3.558201,Democrat,35-44
563,0.0,1.0,4.0,4.0,6.0,1.0,41.0,4.0,19.0,0.0,-2.302585,Democrat,35-44
568,71.0,2.0,4.0,6.0,2.0,0.0,57.0,4.0,19.0,0.0,4.264087,Democrat,55-64
570,83.0,4.0,1.0,4.0,6.0,2.0,43.0,6.0,19.0,0.0,4.420045,Independent,35-44


## 4. Calculating From Data Exercise

For each of the below match-ups, choose the group that is more likely to vote for Bill Clinton. You can calculate this using the percentage of each group that intends to vote for Clinton (`vote`). Which match-up was the closest? Which had the biggest difference?

Another way to think about this: Given that a respondent is a Democrat, there is a ____ percent chance they will vote for Clinton. How does this value change if the respondent is a Republican?

*   Democrats or Republicans
*   People younger than 44 or People 44 and older
*   People who watch TV news at least 6 days a week or People who watch TV news less than 3 days a week
*   People who live somewhere with a population greater than the average respondent or People who live in a place with a population equal to or less than the average respondent


In [7]:
# Your code here
df.head()

,popul,TVnews,selfLR,ClinLR,DoleLR,PID,age,education,income,vote,logpopul,party,age_group
0,0.0,7.0,7.0,1.0,6.0,6.0,36.0,3.0,1.0,1.0,-2.302585,Republican,35-44
1,190.0,1.0,3.0,3.0,5.0,1.0,20.0,4.0,1.0,0.0,5.247550,Democrat,18-24
2,31.0,7.0,2.0,2.0,6.0,1.0,24.0,6.0,1.0,0.0,3.437208,Democrat,18-24
3,83.0,4.0,3.0,4.0,5.0,1.0,28.0,6.0,1.0,0.0,4.420045,Democrat,28-34
4,640.0,7.0,5.0,6.0,4.0,0.0,68.0,6.0,1.0,0.0,6.461624,Democrat,65 and older


## 5. Grouping Data Exercise

Use the `groupby()` method to bucket respondents by `age_group`. Which age group is the most conservative? Which watches TV news the least?

Next, calculate 5 percentile groups based on income. Group the dataset by these percentiles. Which income bracket is the most liberal? Which is the most conservative? The oldest? Highest educated?

In [19]:
# Your code here
grouped=df.groupby('age_group').mean()
print(grouped['selfLR'].sort_values(ascending = False))
print(grouped['TVnews'].sort_values(ascending = True))



age_group
55-64           4.532258
65 and older    4.529412
45-54           4.285714
35-44           4.257143
28-34           4.217391
18-24           4.000000
Name: selfLR, dtype: float64
age_group
18-24           2.358491
28-34           2.570652
35-44           3.118367
45-54           3.916667
55-64           4.516129
65 and older    5.523529
Name: TVnews, dtype: float64


<ipython-input-19-d7469218882e>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped=df.groupby('age_group').mean()


## 6. Voting Across the Aisle

We are interested in learning more about respondents who's political views differ strongly from the candidate they expect to vote for. Using `selfLR`, `vote`, `ClinLR`, and `DoleLR`, work through the following questions. Your interpretation may differ from the answer key.

*   What is the largest recorded difference between a respondent's political leaning and their impression of their intended candidate's political leaning?
*   How many respondents exhibit a difference of that magnitude?
*   Make a separate DataFrame called `sway` that only includes voters who exhibit a difference greater than |3|.
*   Among those in `sway`, are respondents more likely to be voting for a candidate more conservative or more liberal than their own political leaning?
*   In `sway`, which candidate is the more popular choice?



In [ ]:
# Your code here
#  vote - Expected vote
              #  0 - Clinton
               # 1 - Dole
            #The following 3 variables all take the values:
             #   1 - Extremely liberal
              #  2 - Liberal
               # 3 - Slightly liberal
                #4 - Moderate
                #5 - Slightly conservative
                #6 - Conservative
                #7 - Extremely Conservative
            #selfLR - Respondent's self-reported political leanings from "Left"
             #   to "Right".
            #ClinLR - Respondents impression of Bill Clinton's political
             #   leanings from "Left" to "Right".
            #DoleLR  - Respondents impression of Bob Dole's political leanings
             #   from "Left" to "Right".





# BSD 3-Clause License

*Copyright (c) 2022, UC Berkeley School of Information*

*All rights reserved.*

*Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:*

*1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.*

*2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.*

*3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.*

*THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.*